#UK Energy Production with Bayesian LSTMs in PyTorch

Ce notebook est une adaptation de https://colab.research.google.com/drive/1XSouHjY0ImMKNDYqIZtHPnDs_MESinwk#scrollTo=3FQQioKQ-Tkr écrit par Pawarit Laosunthara



# **Important Note for GitHub Readers:**

The original notebook demonstrates an implementation of an (Approximate) Bayesian Recurrent Neural Network in PyTorch, originally inspired by the *Deep and Confident Prediction for Time Series at Uber* (https://arxiv.org/pdf/1709.01907.pdf)

<br>

In this approach, Monte Carlo dropout is used to **approximate** Bayesian inference, allowing our predictions to have explicit uncertainties and confidence intervals. This property makes Bayesian Neural Networks highly appealing to critical applications requiring uncertainty quantification.
The *Appliances energy prediction* dataset used in this example is from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction)



# Preliminary Data Wrangling

**Selected Columns:**

For simplicity and speed when running this notebook, only temporal and autoregressive features are used.


In [1]:
import pandas as pd

In [3]:
path_data = ...
Energy_month = pd.read_csv(path_data+'/energy_sum_month.csv', index_col='timestamp',parse_dates=True)

In [4]:
Energy_month.head()

,coal,nuclear,wind,hydro,solar,total
timestamp,,,,,,
2012-01-31,1.557582e+08,6.846972e+07,1.539371e+07,6.375348e+06,0.0,2.459970e+08
2012-02-29,1.690382e+08,6.966962e+07,1.363287e+07,4.662923e+06,0.0,2.570037e+08
2012-03-31,1.644467e+08,5.640124e+07,1.025568e+07,3.784780e+06,0.0,2.348884e+08
2012-04-30,1.384968e+08,6.967671e+07,9.365615e+06,1.832103e+06,0.0,2.193712e+08
2012-05-31,1.207326e+08,7.079185e+07,8.451584e+06,1.891290e+06,0.0,2.018673e+08


## Time Series Transformations

1. The dataset is to be re-sampled at an hourly rate for more meaningful analytics.

2. To alleviate exponential effects, the target variable is log-transformed as per the Uber paper.

3. For simplicity and speed when running this notebook, only temporal and autoregressive features, namely `day_of_month` \
and previous values of `energy_porduction` are used as features

In [8]:
resample_df = Energy_month['total'].reset_index()
resample_df.columns = ['date', 'energy_production']
resample_df

,date,energy_production
0,2012-01-31,2.459970e+08
1,2012-02-29,2.570037e+08
2,2012-03-31,2.348884e+08
3,2012-04-30,2.193712e+08
4,2012-05-31,2.018673e+08
...,...,...
86,2019-03-31,1.300339e+08
87,2019-04-30,1.169793e+08
88,2019-05-31,1.012899e+08
89,2019-06-30,9.308057e+07


In [9]:
import numpy as np


resample_df['day_of_month'] = resample_df['date'].dt.month.astype(int)


datetime_columns = ['date', 'day_of_month']
target_column = 'energy_production'

feature_columns = datetime_columns + ['energy_production']

# For clarity in visualization and presentation, 
# only consider the first 150 hours of data.
resample_df = resample_df[feature_columns]

In [10]:
resample_df

,date,day_of_month,energy_production
0,2012-01-31,1,2.459970e+08
1,2012-02-29,2,2.570037e+08
2,2012-03-31,3,2.348884e+08
3,2012-04-30,4,2.193712e+08
4,2012-05-31,5,2.018673e+08
...,...,...,...
86,2019-03-31,3,1.300339e+08
87,2019-04-30,4,1.169793e+08
88,2019-05-31,5,1.012899e+08
89,2019-06-30,6,9.308057e+07


In [11]:
import plotly.express as px

plot_length = 150
plot_df = resample_df.copy(deep=True).iloc[:plot_length]
plot_df['day_of_month'] = plot_df['date'].dt.month_name()

fig = px.line(plot_df,
              x="date",
              y="energy_production", 
              color="day_of_month", 
              title="Log of UK Energy Production vs Time")
fig.show()

# Prepare Training Data

For this example, we will use sliding windows of 10 points per each window (equivalent to 10 hours) to predict each next point. The window size can be altered via the `sequence_length` variable.

Min-Max scaling has also been fitted to the training data to aid the convergence of the neural network. 

In [13]:
from sklearn.preprocessing import MinMaxScaler

def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)

train_split = 0.73
n_train = int(train_split * len(resample_df))
n_test = len(resample_df) - n_train

features = ['day_of_month', 'energy_production']
feature_array = resample_df[features].values

# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=features)

sequence_length = 12
X, y = create_sliding_window(scaled_array, 
                             sequence_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

# Define Bayesian LSTM Architecture

To demonstrate a simple working example of the Bayesian LSTM, the model as defined in Uber's paper has been used a starting point. The network architecture is as follows:

Encoder-Decoder Stage:
 - 1 uni-directional LSTM layer with 128 hidden units acts as an encoding layer to construct a fixed-dimension embedding state
 - 1 uni-directional LSTM layer with 32 hidden units acts as a decoding layer to produce predictions at future steps
 - Dropout is applied at **both** training and inference for both LSTM layers


 Predictor Stage:
 - 1 fully-connected output layer with 1 output (for predicting the target value) to produce a single value for the target variable


By allowing dropout at both training and testing time, the model simulates random sampling, thus allowing varying predictions that can be used to estimate the underlying distribution of the target value, enabling explicit model uncertainties.


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length):

        super(BayesianLSTM, self).__init__()

        self.hidden_size_1 = 96 #128
        self.hidden_size_2 = 24 #32
        self.n_layers = 1 # number of (stacked) LSTM layers for each stage

        self.lstm1 = nn.LSTM(n_features, 
                             self.hidden_size_1, 
                             num_layers=1,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=1,
                             batch_first=True)
        
        self.dense = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=0.5, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=0.5, training=True)
        output = self.dense(state[0].squeeze(0))
        
        return output
        
    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state
    
    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

### Begin Training

To train the Bayesian LSTM, we use the ADAM optimizer along with mini-batch gradient descent (`batch_size = 128`). For quick demonstration purposes, the model is trained for 150 epochs.

The Bayesian LSTM is trained on the first 73% of data points, using the aforementioned sliding windows of size 12. The remaining 27% of the dataset is held out purely for testing. These ratios are choosen to take one year (12 last months) to test. 

In [15]:
n_features = scaled_array.shape[-1]
sequence_length = 12
output_length = 1

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=0.001)

batch_size = 24
n_epochs = 1000

In [16]:
bayesian_lstm.train()
loss_results = []
epochs_results = []

for e in range(1, n_epochs+1):
    for b in range(0,len(X_train), batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size]    

        X_batch = torch.tensor(features,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch) 
        loss = criterion(output.view(-1), y_batch)  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 

    if e % 10 == 0:
      print('epoch: ', e, 'loss: ', loss.item())
      loss_results.append(loss.item())
      epochs_results.append(e)

epoch:  10 loss:  0.027458924800157547
epoch:  20 loss:  0.026334933936595917
epoch:  30 loss:  0.016699058935046196
epoch:  40 loss:  0.01140481885522604
epoch:  50 loss:  0.006369076203554869
epoch:  60 loss:  0.005720851477235556
epoch:  70 loss:  0.003908832091838121
epoch:  80 loss:  0.0034340964630246162
epoch:  90 loss:  0.005362594034522772
epoch:  100 loss:  0.005086849443614483
epoch:  110 loss:  0.004946643486618996
epoch:  120 loss:  0.004322368651628494
epoch:  130 loss:  0.004986060783267021
epoch:  140 loss:  0.00517632020637393
epoch:  150 loss:  0.005645728670060635
epoch:  160 loss:  0.004691746085882187
epoch:  170 loss:  0.005045131780207157
epoch:  180 loss:  0.004933128133416176
epoch:  190 loss:  0.005374653730541468
epoch:  200 loss:  0.004095863085240126
epoch:  210 loss:  0.003183485707268119
epoch:  220 loss:  0.0035038511268794537
epoch:  230 loss:  0.004046110901981592
epoch:  240 loss:  0.003798526246100664
epoch:  250 loss:  0.003687727265059948
epoch:  2

In [ ]:
#Plot the loss function
loss_df = pd.DataFrame({'epochs':epochs_results, 'loss':loss_results})
fig = px.line(loss_df,
                 x="epochs",
                 y="loss",
                 title="Loss as function of epochs")
fig.show()


# Evaluating Model Performance

The Bayesian LSTM implemented is shown to produce reasonably accurate and sensible results on both the training and test sets, often comparable to other existing frequentist machine learning and deep learning methods.



In [17]:
offset = sequence_length

def inverse_transform(y):
  return target_scaler.inverse_transform(y.reshape(-1, 1))

training_df = pd.DataFrame()
training_df['date'] = resample_df['date'].iloc[offset:n_train + offset:1] 
training_predictions = bayesian_lstm.predict(X_train)
training_df['energy_production'] = inverse_transform(training_predictions)
training_df['source'] = 'Training Prediction'

training_truth_df = pd.DataFrame()
training_truth_df['date'] = training_df['date']
training_truth_df['energy_production'] = resample_df['energy_production'].iloc[offset:n_train + offset:1] 
training_truth_df['source'] = 'True Values'

testing_df = pd.DataFrame()
testing_df['date'] = resample_df['date'].iloc[n_train + offset::1] 
testing_predictions = bayesian_lstm.predict(X_test)
testing_df['energy_production'] = inverse_transform(testing_predictions)
testing_df['source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['date'] = testing_df['date']
testing_truth_df['energy_production'] = resample_df['energy_production'].iloc[n_train + offset::1] 
testing_truth_df['source'] = 'True Values'

evaluation = pd.concat([training_df, 
                        testing_df,
                        training_truth_df,
                        testing_truth_df
                        ], axis=0)

In [19]:
fig = px.line(evaluation.loc[evaluation['date'].between('2012', '2020')],
                 x="date",
                 y="energy_production",
                 color="source",
                 title="Log of UK energy production in Wh vs Time")
fig.show()


Test
MAE :   9803512.165973967
RMSE :   13004292.108600525
Train
MAE :   5887732.665595272
RMSE :   8012573.391926678


In [ ]:
#Print MAE & RMSE for training and testing dataset
print('Test')
print('MAE :  ', np.sum(abs(testing_truth_df['energy_production'] - testing_df['energy_production']))/len(testing_truth_df['energy_production']))
print('RMSE :  ', np.sqrt(np.sum((testing_truth_df['energy_production'] - testing_df['energy_production'])**2)/len(testing_truth_df['energy_production'])))
print('Train')
print('MAE :  ', np.sum(abs(training_truth_df['energy_production'] - training_df['energy_production']))/len(training_truth_df['energy_production']))
print('RMSE :  ', np.sqrt(np.sum((training_truth_df['energy_production'] - training_df['energy_production'])**2)/len(training_truth_df['energy_production'])))

Test
MAE :   9997836.473666275
RMSE :   12447024.716353463
Train
MAE :   6430481.875348846
RMSE :   8292199.701449063


# Uncertainty Quantification

The fact that stochastic dropouts are applied after each LSTM layer in the Bayesian LSTM enables users to interpret the model outputs as random samples from the posterior distribution of the target variable. 

This implies that by running multiple experiments/predictions, can approximate  parameters of the posterioir distribution, namely the mean and the variance, in order to create confidence intervals for each prediction.

In this example, we construct 99% confidence intervals that are three standard deviations away from the approximate mean of each prediction.

In [20]:
n_experiments = 200

test_uncertainty_df = pd.DataFrame()
test_uncertainty_df['date'] = testing_df['date']

for i in range(n_experiments):
  experiment_predictions = bayesian_lstm.predict(X_test)
  test_uncertainty_df['log_energy_consumption_{}'.format(i)] = inverse_transform(experiment_predictions)

log_energy_consumption_df = test_uncertainty_df.filter(like='log_energy_consumption', axis=1)
test_uncertainty_df['log_energy_consumption_mean'] = log_energy_consumption_df.mean(axis=1)
test_uncertainty_df['log_energy_consumption_std'] = log_energy_consumption_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['date', 'log_energy_consumption_mean', 'log_energy_consumption_std']]

In [21]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['log_energy_consumption_mean'] - 3*test_uncertainty_df['log_energy_consumption_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['log_energy_consumption_mean'] + 3*test_uncertainty_df['log_energy_consumption_std']

In [23]:
import plotly.graph_objects as go

test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['date'].between('2012', '2020')]
truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['date'].between('2012', '2020')]

upper_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )
lower_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.5)',
    name='99% Lower Confidence Bound'
    )
real_trace = go.Scatter(
    x=truth_uncertainty_plot_df['date'],
    y=truth_uncertainty_plot_df['energy_production'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

data = [upper_trace, lower_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for Energy Consumption Test Data',
                   xaxis_title='Time',
                   yaxis_title='Total UK energy production')

fig.show()

#### Evaluating Uncertainty

Using multiple experiments above, 99% confidence intervals have been constructed for each the prediction of the target variable (the logarithm of appliance power consumption). While we can visually observe that the model is generally capturing the behavior of the time-series, approximately only 50% of the real data points lie within a 99% confidence interval from the mean prediction value.

Despite the relatively low percentage of points within the confidence interval, it must be noted that Bayesian Neural Networks only seek to quantify the epistemic model uncertainty and does not account for aleatoric uncertainty (i.e. noise).

In [25]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['log_energy_consumption_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['energy_production']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:", 
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.5384615384615384
